In [1]:
import pinocchio as pin
import math
import time
from pinocchio.robot_wrapper import RobotWrapper
from pinocchio.utils import *
import pinocchio as pio
import eigenpy
from scipy.optimize import fmin_bfgs
import numpy as np

import casadi as ca

from pinocchio import casadi as cpin

# from pinocchio.visualize import GepettoVisualizer

import matplotlib.pyplot as plt

In [2]:
exampleRobotDataPath = '/home/n7student/Documents/Boulot/CNRS@CREATE/Codes/OC & IRL/Starting/3 - URDF et pinocchio/assets/'
urdf = exampleRobotDataPath + 'mon_robot.urdf'
robot = RobotWrapper.BuildFromURDF( urdf, [ exampleRobotDataPath, ] )
# robot.setVisualizer(GepettoVisualizer()) # AJOUT
robot.initViewer(loadModel=True)
NQ = robot.model.nq
NV = robot.model.nv

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
cmodel = cmodel = cpin.Model(robot.model)
cdata = cdata = cmodel.createData()

q = ca.SX.sym("q",NQ,1)
dq = ca.SX.sym("dq",NQ,1)
ddq = ca.SX.sym("ddq",NQ,1)     # NORMALEMENT PAS BESOIN

In [4]:
# Implémentation des fonctions nécessaires au calcul des coûts avec cpin

frame_id = cmodel.getFrameId("ee_link")

# def vtip_fun(q):
    # cpin.forwardKinematics(cmodel, cdata, q)
    # v_frame = cpin.getFrameVelocity(cmodel, cdata, frame_id, cpin.ReferenceFrame.LOCAL_WORLD_ALIGNED)
    # NOTE : vérifier que ce sont bien les deux bonnes composantes de la vitesse
    # return v_frame.linear[[0, 2]]
    # return cpin.computeFrameJacobian(cmodel, cdata, q, frame_id, cpin.ReferenceFrame.LOCAL_WORLD_ALIGNED)[:3,:]

vtip = ca.Function('vtip', [q], [cpin.computeFrameJacobian(cmodel, cdata, q, frame_id, cpin.ReferenceFrame.LOCAL_WORLD_ALIGNED)[:3,:]])

vtip_val = vtip(q) 
vx_sq = ca.dot(vtip_val[0], vtip_val[0])
vy_sq = ca.dot(vtip_val[1], vtip_val[1])

vx_sq_fun = ca.Function("vx_sq_fun", [q], [vx_sq])
vy_sq_fun = ca.Function("vy_sq_fun", [q], [vy_sq])

# oMtool = robot.data.oMf[IDX_EE]
# oRtool = oMtool.rotation
# Jee = oRtool[:2,:2] @ (pin.computeFrameJacobian(robot.model,robot.data,np.array([q1_test[i],q2_test[i]]),IDX_EE)[:2,:2])

jac_fun = ca.Function("jac_fun", [q], [cpin.computeFrameJacobian(cmodel, cdata, q, frame_id, cpin.ReferenceFrame.LOCAL_WORLD_ALIGNED)])

manipulability_fun = ca.Function("manipulability_fun", [q], [jac_fun(q) @ jac_fun(q).T])

cost_manipulability_fun = ca.Function("cost_manipulability_fun", [q], [-ca.log(ca.det(manipulability_fun(q)) + 1e-6)])

In [5]:
opti = ca.Opti()

# cadre du problème
# temporel
t_0 = 0
t_f = 1
N = 100  # nombre de points de discrétisation
dt = (t_f - t_0) / N

# longueurs
L_1 = 1
L_2 = 1
N_angles = 2

# variables à optimiser
q = opti.variable(N_angles, N)
q_fic = opti.variable(1, N)
dq = opti.variable(N_angles, N)

# déclaration de la matrice de poids
# NOTE : j'avais mis des np.ones au début, mais cela ne marchait pas, il faut essayer de rester avec des objets casadi
w_1 = ca.DM.ones((N_angles, N))
w_2 = ca.DM.ones(N)
w_3 = ca.DM.zeros(N)
w_4 = ca.DM.zeros(N)


# \Phi_1 : coût sur les vitesses angulaires
Phi_1 = dq**2
J_1 = w_1[0,:] @ Phi_1[0,:].T + w_1[1,:] @ Phi_1[1,:].T

# variables symboliques indépendantes pour un pas de temps
q_sym = ca.MX.sym("q", 2)


J_2 = 0
J_3 = 0
J_4 = 0
save_cost_manipulability = []
for t in range(N):
    q_t = ca.vertcat(q[0, t], q[1, t])
    # q_t = ca.vertcat(q[0, t], q[1, t], q_fic[0, t])
    # cpin.framesForwardKinematics(cmodel, cdata, q_t)
    J_2 += w_2[t] * vx_sq_fun(q_t)
    J_3 += w_3[t] * vy_sq_fun(q_t)
    J_4 += w_4[t] * cost_manipulability_fun(q_t)

# fonction que l'on cherche à minimiser
opti.minimize(J_1 + J_2 + J_3 + J_4)

# contraintes d’intégration
opti.subject_to(q_fic[0, N-1] == 0)
for t in range(N-1):
    opti.subject_to(q[:, t+1] == q[:, t] + dt * dq[:, t])
    opti.subject_to(q_fic[0, t] == 0)
opti.subject_to(q[:, 0] == [0, ca.pi/4])
opti.subject_to(L_1*ca.cos(q[0, -1]) + L_2*ca.cos(q[0, -1] + q[1, -1]) == -0.75)

# conditions supplémentaires sur les angles, non demandé dans l'énoncé
# opti.subject_to(opti.bounded(-pi/2, q, pi/2))

opti.solver("ipopt")
sol = opti.solve()

q1_cpin = sol.value(q[0,:])
q2_cpin = sol.value(q[1,:])

dq1_cpin = sol.value(dq[0,:])
dq2_cpin = sol.value(dq[1,:])

print("q1 =", q1_cpin)
print("q2 =", q2_cpin)
print("dq1 =", dq1_cpin)
print("dq2 =", dq2_cpin)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      698
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      500

Total number of variables............................:      500
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      301
Total number of inequality c

In [6]:
robot.display(np.array([q1_cpin[0], q2_cpin[0]]))

time.sleep(2)
for i in range(N):
    robot.display(np.array([q1_cpin[i],q2_cpin[i]]))
    time.sleep(0.1)

In [14]:
for i in range(N):
    print(jac_fun([q1_cpin[i],q2_cpin[i]]))


[[-0.707107, -0.707107], 
 [1.70711, 0.707107], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.702658, -0.718599], 
 [1.6953, 0.695425], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.698021, -0.729898], 
 [1.68305, 0.683556], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.693196, -0.741001], 
 [1.67036, 0.671504], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.688183, -0.751906], 
 [1.65724, 0.659271], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.682986, -0.762608], 
 [1.64369, 0.646861], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.677603, -0.773105], 
 [1.62971, 0.634278], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.672037, -0.783395], 
 [1.6153, 0.621524], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.66629, -0.793475], 
 [1.60048, 0.608603], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.660361, -0.803341], 
 [1.58524, 0.595519], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.654253, -0.812992], 
 [1.5696, 0.582275], 
 [0, 0], 
 [0, 0], 
 [0, 0], 
 [1, 1]]

[[-0.647967, -0.822424],

In [9]:
results_angle = np.array([q1_cpin, q2_cpin])

In [10]:
results_angle.tofile(f"opti_angles_cpin_w1_{w_1[0]}_w2_{w_2[0]}_w3_{w_3[0]}_w4_{w_4[0]}.txt")